In [96]:
import boto3 # Python integration with AWS.
import pandas as pd # Read csv.
from io import StringIO, BytesIO # StringIO to read csv on AWS. BytesIO to save as Parquet file to S3.
from datetime import datetime, timedelta # Date functions.

## Functions

In [97]:
# Adapter Layer Functions

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    # Read csvs with StringIO (needed for AWS) and Pandas
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

# Update meta csv file
def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [98]:
# Application Layer Functions, ETL

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    # DF with filtered columns.
    df = df.loc[:, columns]
    # Filter out empty lines.
    df.dropna(inplace=True)
    # Get opening price per ISIN and day
    # Create new column with opening price.
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    # Get closing price per ISIN and day
    # Create new column with closing price.
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    # Aggregation
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    # Percent Change Prev Closing
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    # Remove column and update df
    df.drop(columns=['prev_closing_price'], inplace=True)
    # Round everything.
    df = df.round(decimals=2)
    # Filter by date, don't show previous date that was used for calculation.
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format, meta_key, extract_date_list):
    # Write to S3.
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

# Combine all three ETL functions.
def etl_report1(scr_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format, meta_key):
    df = extract(scr_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(trg_bucket, df, trg_key, trg_format, meta_key, extract_date_list)
    return True

In [99]:
# Application layer - not core

# Restructure get_objects
def return_date_list(bucket, arg_date, scr_format, meta_key):
    # Convert to date type and use previous day
    min_date = datetime.strptime(arg_date, scr_format).date() - timedelta(days=1)
    today = datetime.today().date()
    
    # Using meta_file.csv        
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        # Dates list from meta file
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        # Min date
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(scr_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(scr_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').execptions.NoSuchKey:
   

        # Return date list in string format if meta file is empty
        return_dates = [(min_date + timedelta(days=x)).strftime(scr_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)

In [100]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    # Argument Date for start of report
    arg_date = '2022-11-26'
    # Moving all arguments and parameters to the beggining of the code
    scr_format = '%Y-%m-%d'
    scr_bucket = 'xetra-1234'
    trg_bucket = 'manoetl123'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    # Init
    # Variables for data on AWS S3.
    s3 = boto3.resource('s3')
    bucket_scr = s3.Bucket(scr_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, scr_format, meta_key)
    etl_report1(bucket_scr, bucket_trg, date_list, columns, extract_date, trg_key, trg_format, meta_key)

In [101]:
# run

main()

## Read the uploaded file

In [102]:
trg_bucket = 'manoetl123'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)

# Read the uploaded file
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20221122_222801.parquet
xetra_daily_report_20221122_233442.parquet
xetra_daily_report_20221123_183552.parquet
xetra_daily_report_20221124_210807.parquet
xetra_daily_report_20221127_195456.parquet
xetra_daily_report_20221127_200047.parquet
xetra_daily_report_20221127_201807.parquet


In [105]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20221127_201807.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [106]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-11-26,36.10,37.70,36.10,37.70,2864,0.0
1,AT000000STR1,2022-11-27,36.10,37.70,36.10,37.70,2864,0.0
2,AT00000FACC2,2022-11-26,7.90,8.36,7.86,8.36,2490,0.0
3,AT00000FACC2,2022-11-27,7.90,8.36,7.86,8.36,2490,0.0
4,AT0000606306,2022-11-26,16.88,17.15,16.50,17.86,115948,0.0
...,...,...,...,...,...,...,...,...
6289,XS2314660700,2022-11-27,22.37,21.86,21.86,22.37,598,0.0
6290,XS2376095068,2022-11-26,34.28,34.90,34.28,35.08,1360,0.0
6291,XS2376095068,2022-11-27,34.28,34.90,34.28,35.08,1360,0.0
6292,XS2434891219,2022-11-26,3.44,3.50,3.44,3.50,0,0.0
